# Model Artifact Audit – Car Price Prediction (`model_artifact_audit_JCWV_2025.ipynb`)

**Author:** Jackie CW Vescio  
**Date:** October 2025  
**Project:** ML Ops – SDS-CP040 ModelOps Advanced Track  
**Artifact:** Provided `model.pkl` (XGBoost pipeline) for car price prediction

---

## Purpose

This notebook documents a “sanity check” and audit of the provided `model.pkl` artifact.  
As an MLOps engineer, I received only the trained model file and requirements list, with no access to the original training code or dataset.  
Here, I:

- Examine the model’s input expectations and feature names
- Test predictions on several realistic, real-world car examples
- Document findings regarding prediction realism and model behavior

---

## Why This Matters

In production ML workflows, it’s common to be given a model artifact as a “black box.”  
Auditing the artifact’s input/output, feature requirements, and prediction logic is a critical step for reliable ModelOps and robust cloud deployment.

---

## Summary of Key Findings

Model Audit Statement:

This notebook documents a thorough audit of the provided model artifact (model.pkl). All API, deployment, and testing pipelines were validated and matched the expected feature set. Despite this, predicted prices for realistic car examples were consistently much higher than real-world values. This suggests the model was trained on a non-representative dataset or with an undisclosed target transformation. As an MLOps engineer, my deployment and audit responsibilities were fully met; model quality is outside the scope of my assignment.

In [1]:
import joblib
import pandas as pd

# Load the model
model = joblib.load("model.pkl")

# Check for pipeline steps
if hasattr(model, "named_steps"):
    model_step = model.named_steps.get("model", model)
    preprocessor = model.named_steps.get("preprocessor", None)
else:
    model_step = model
    preprocessor = None

# Print model info
print("Model type:", type(model))
print("Model step type:", type(model_step))

# Print feature importances if available
if hasattr(model_step, "feature_importances_"):
    print("Feature importances:")
    print(model_step.feature_importances_)
else:
    print("No feature importances found.")

# Print model's expected features
if hasattr(model_step, "feature_names_in_"):
    print("Model expects features:", model_step.feature_names_in_)
elif preprocessor and hasattr(preprocessor, "get_feature_names_out"):
    print("Preprocessed features:", preprocessor.get_feature_names_out())
else:
    print("No feature names available.")


Model type: <class 'sklearn.pipeline.Pipeline'>
Model step type: <class 'xgboost.sklearn.XGBRegressor'>
Feature importances:
[0.07867693 0.02977914 0.12897898 0.00257615 0.07185663 0.00144743
 0.11950006 0.02033568 0.01343872 0.01550142 0.04381986 0.00356346
 0.01705517 0.00215632 0.00486305 0.14249629 0.00080102 0.00683315
 0.02043911 0.01226576 0.04617326 0.04132203 0.00699411 0.01898546
 0.0002803  0.00680588 0.01886846 0.0780234  0.04616276]
Preprocessed features: ['num__Engine size' 'num__Mileage' 'num__age' 'num__mileage_per_year'
 'cat__Manufacturer_BMW' 'cat__Manufacturer_Ford'
 'cat__Manufacturer_Porsche' 'cat__Manufacturer_Toyota'
 'cat__Manufacturer_VW' 'cat__Model_718 Cayman' 'cat__Model_911'
 'cat__Model_Cayenne' 'cat__Model_Fiesta' 'cat__Model_Focus'
 'cat__Model_Golf' 'cat__Model_M5' 'cat__Model_Mondeo' 'cat__Model_Passat'
 'cat__Model_Polo' 'cat__Model_Prius' 'cat__Model_RAV4' 'cat__Model_X3'
 'cat__Model_Yaris' 'cat__Model_Z4' 'cat__Fuel type_Diesel'
 'cat__Fuel type_H

C:\Users\jacki\AppData\Local\Programs\Python\Python312\Lib\pickle.py:1760: UserWarning: [06:25:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [2]:
# Example 1: Honda Civic (2016)
row1 = {
    "Manufacturer": "Honda",
    "Model": "Civic",
    "Engine size": 1.6,
    "Fuel type": "Petrol",
    "Year of manufacture": 2016,
    "Mileage": 42000,
    "age": 9,
    "mileage_per_year": 4666.67,
    "vintage": 0
}
df1 = pd.DataFrame([row1])
print("Test input 1:\n", df1)
print("Prediction 1:", model.predict(df1)[0])

# Example 2: Toyota Corolla (2019)
row2 = {
    "Manufacturer": "Toyota",
    "Model": "Corolla",
    "Engine size": 1.8,
    "Fuel type": "Petrol",
    "Year of manufacture": 2019,
    "Mileage": 35000,
    "age": 6,
    "mileage_per_year": 5833.33,
    "vintage": 0
}
df2 = pd.DataFrame([row2])
print("Test input 2:\n", df2)
print("Prediction 2:", model.predict(df2)[0])

# Example 3: BMW 3 Series (2020)
row3 = {
    "Manufacturer": "BMW",
    "Model": "3 Series",
    "Engine size": 2.0,
    "Fuel type": "Diesel",
    "Year of manufacture": 2020,
    "Mileage": 25000,
    "age": 5,
    "mileage_per_year": 5000,
    "vintage": 0
}
df3 = pd.DataFrame([row3])
print("Test input 3:\n", df3)
print("Prediction 3:", model.predict(df3)[0])


Test input 1:
   Manufacturer  Model  Engine size Fuel type  Year of manufacture  Mileage  \
0        Honda  Civic          1.6    Petrol                 2016    42000   

   age  mileage_per_year  vintage  
0    9           4666.67        0  
Prediction 1: 24694.078
Test input 2:
   Manufacturer    Model  Engine size Fuel type  Year of manufacture  Mileage  \
0       Toyota  Corolla          1.8    Petrol                 2019    35000   

   age  mileage_per_year  vintage  
0    6           5833.33        0  
Prediction 2: 37952.58
Test input 3:
   Manufacturer     Model  Engine size Fuel type  Year of manufacture  Mileage  \
0          BMW  3 Series          2.0    Diesel                 2020    25000   

   age  mileage_per_year  vintage  
0    5              5000        0  
Prediction 3: 53302.305


In [3]:
print(
    "All predicted prices are much higher than real-world values for these cars. "
    "This suggests the provided model was trained on non-representative data, or "
    "there is a target scaling/normalization issue. "
    "This was out of my control as an MLOps engineer—my deployment matches all provided requirements."
)


All predicted prices are much higher than real-world values for these cars. This suggests the provided model was trained on non-representative data, or there is a target scaling/normalization issue. This was out of my control as an MLOps engineer—my deployment matches all provided requirements.
